# Data Augmentation

Альбументация данных - это такой подход, который используется для расширения обучающей выборки за счёт небольших изменений исходных данных. Например, сдвига данных, отражения по вертикальной/горизонтальной оси, изменение насыщенности пикселей (в случае картинок). 

Помимо увеличения количества данных в случае, когда их исходно мало, альбументация данных помогает избежать переобучения модели, а также помогает увеличить устойчивость работы модели (так как при обучении она видит разные модификации данных). 

Альбументацию можно применять к разным типам данных: 
- аудио
- текст
- изображения
  

Для альбументации изображений можно использовать:
- геометрические преобразования - рандомная обрезка, поворот, отражение и т.д. 
- изменение цветового пространства - изменение интенсивности, яркости пикселей и т.д.
- фильтрация - размытие, изменение резкости и т.д.
- рандомная зачистка - удаление части исходной картинки 

Для текста можно использовать: 
- перемешивание слов/предложений 
- перестановка слов - замена слов синонимами 
- манипуляция с текстом - перефразирование предложений

Для аудио можно использовать: 
- добавление шумов 
- смещение каналов 
- изменение скорости 

Это только малая часть альбументаций, которую можно использовать. На самом деле их существует намного и намного больше. 


Давайте посмотрим наглядно, что такое альбументации. Использовать будем датасет MNIST, весь датасет нам не понадобится, только пара картинок оттуда. 

In [ ]:
%load_ext autoreload
%autoreload 2

import albumentations as albu
import cv2
import torchvision
from torchvision.transforms import transforms
import matplotlib.pyplot as plt 
import numpy as np 
from ipywidgets import interact, IntSlider

RANDOM_SEED = 42

Загрузим обучающую выборку в папку data. Сами данные преобразуем в список, чтобы дальше было удобнее работать. 

In [ ]:
src_data = list(
    torchvision.datasets.MNIST(
        "data", train=True, transform=None, download=True
    )
)

In [ ]:
images, labels = [], []
for info in src_data:
    img, label = info 
    images.append(np.array(img))
    labels.append(label)

print(f"Number of images: {len(images)}")
print(f"Number of labels: {len(labels)}")

Все картинки нам, конечно, не понадобится, возьмём только пару из них для наглядности.

In [ ]:
plt.figure(figsize=(10,5))
for i in range(10):
    plt.subplot(2,5,i+1)
    plt.imshow(images[i], cmap='gray')
plt.show()

Давайте попробуем применить к нашим картинкам сдвиг, для наглядности заполним белым цветом пиксели, которые появятся в результате сдвига. Но в реальных условиях лучше выбирать цвет исходя из данных. 

Например, у нас в данных фон чёрный, логичнее было бы заполнять недостающие пиксели чёрным. 

Важно также упомянуть, что параметр `p` отвечает за вероятность применения альбументации, в нашем примере он выставлен в `p = 0.9`, чтобы как можно чаще получать изменённую картинку, в реальной задаче такая частота изменения может пагубно сказаться на обучении модели, так что лучше устанавливать значение поменьше. 

In [ ]:
transform = albu.ShiftScaleRotate(
    shift_limit=0.3,
    scale_limit=0,
    rotate_limit=0,
    interpolation=3,
    border_mode=cv2.BORDER_CONSTANT,
    p=0.9,
    value=255,  # white background for better representation
)

@interact
def show(ind=IntSlider(val=0, min=0, max=len(images)-1)):
    _, ax = plt.subplots(nrows=1, ncols=2, figsize=(8, 16))

    img = images[ind]
    transformed_img = transform(image=img)["image"]

    ax[0].imshow(img, cmap="gray")
    ax[0].set_title("Исходное изображение")

    ax[1].imshow(transformed_img, cmap="gray")
    ax[1].set_title("Изменённое изображение")

    plt.show()

Сейчас у нас параметры `scale_limit` и `rotate_limit` выставлены в 0, но если установить там значения, то к изображению будут применяться сразу 3 альбументации: сдвиг, изменение масштаба и поворот. 

Также можно использовать отражение изображения по вертикальной или/и горизонтальной оси. 

Однако с этим нужно быть аккуратным, т.к. в задаче распознвания цифр, например, такая альбументация может сделать только хуже. Посколько при написании цифр люди зачастую не пишут их наоборот, верно. Таким образом, если мы модели покажем отражённые цифры это может ввести её в заблуждение. 

Но мы с вами посмотрим как это выглядит (=

In [ ]:
transform = albu.HorizontalFlip(p=0.9)

@interact
def show(ind=IntSlider(val=0, min=0, max=len(images)-1)):
    _, ax = plt.subplots(nrows=1, ncols=2, figsize=(8, 16))

    img = images[ind]
    transformed_img = transform(image=img)["image"]

    ax[0].imshow(img, cmap="gray")
    ax[0].set_title("Исходное изображение")

    ax[1].imshow(transformed_img, cmap="gray")
    ax[1].set_title("Изменённое изображение")

    plt.show()

Кроме геометрических преобразований можно использовать различные шумы, например, размытие Гаусса. 

Здесь также как и во всех примерах `p = 0.9` - учтите это при реальных разработках. 

In [ ]:
transform = albu.GaussianBlur(blur_limit=3, p=0.9)

@interact
def show(ind=IntSlider(val=0, min=0, max=len(images)-1)):
    _, ax = plt.subplots(nrows=1, ncols=2, figsize=(8, 16))

    img = images[ind]
    transformed_img = transform(image=img)["image"]

    ax[0].imshow(img, cmap="gray")
    ax[0].set_title("Исходное изображение")

    ax[1].imshow(transformed_img, cmap="gray")
    ax[1].set_title("Изменённое изображение")

    plt.show()

Кроме того можно инвертировать цвета, т.е в нашем случае черное станет белым, а белое, наоборот, чёрным.

In [ ]:
transform = albu.InvertImg(p=0.9)

@interact
def show(ind=IntSlider(val=0, min=0, max=len(images)-1)):
    _, ax = plt.subplots(nrows=1, ncols=2, figsize=(8, 16))

    img = images[ind]
    transformed_img = transform(image=img)["image"]

    ax[0].imshow(img, cmap="gray")
    ax[0].set_title("Исходное изображение")

    ax[1].imshow(transformed_img, cmap="gray")
    ax[1].set_title("Изменённое изображение")

    plt.show()

Также можно создать сразу "список" альбументацией, которые будут применяться рандомно.

Конструкция `albu.Oneof`, будет выбирать одну альбументацию из списка и применять к картинке.

In [ ]:
transform = albu.Compose(
    [
        albu.OneOf(
            [
                albu.ShiftScaleRotate(
                    shift_limit=0.5,
                    scale_limit=0,
                    rotate_limit=0,
                    interpolation=3,
                    border_mode=cv2.BORDER_CONSTANT,
                    p=0.9,
                    value=255,  # white background for better representation
                ),
                albu.ShiftScaleRotate(
                    shift_limit=0,
                    scale_limit=0.5,
                    rotate_limit=0,
                    interpolation=3,
                    border_mode=cv2.BORDER_CONSTANT,
                    p=0.9,
                    value=255,  # white background for better representation
                ),
                albu.ShiftScaleRotate(
                    shift_limit=0,
                    scale_limit=0,
                    rotate_limit=50,
                    interpolation=3,
                    border_mode=cv2.BORDER_CONSTANT,
                    p=0.9,
                    value=255,  # white background for better representation
                ),
                albu.InvertImg(p=0.9)
            ]
        )
    ], 
    p=1, 
)

@interact
def show(ind=IntSlider(val=0, min=0, max=len(images)-1)):
    _, ax = plt.subplots(nrows=1, ncols=2, figsize=(8, 16))

    img = images[ind]
    transformed_img = transform(image=img)["image"]

    ax[0].imshow(img, cmap="gray")
    ax[1].imshow(transformed_img, cmap="gray")

    plt.show()

При применении альбументаций, исходное изображение не всегда меняется (это зависит от значения вероятности `p`). Чем ниже это значение, тем больше шансов, что исходное изоображение не будет как-то изменено - это сделано осознанно, т.к. при обучении модели важнее, чтобы она уловила общую зависимость из исходных данных. Если мы будем подавать модели только изменённые данные, то она может начать ошибаться на исходных данных - а нам оно надо? Конечно, нет (= 

Это как, у меня нет ключа, но у меня есть кое-что получше - рисунок ключа (= 

Для применения альбументаций удобно вынести определённый список в отдельный класс и дальше при генерации данных использовать его. 

Можно использовать класс для изменения обучающих данных, чтобы сразу научить модель всяким искажениям в данных и показать ей, как неидеален реальный мир. 

Либо, можно оценить работу модели на данных с альбументациями, чтобы понять насколько модель устойчива к искажением и с какими изменениями она совсем не может справиться. Но в этом случае важно понимать, что нельзя сравнивать метрики, полученные на исходной тестовой выборке (без альбументацие) и с альбументациями, такое сравнение будет некорректно, т.к. по сути наборы данных разные. 

Далее приведён пример класса, но не забывайте, что под каждую задачу, нужно выбирать свой список альбументацией, который больше всего подходит. Успехов! 

In [ ]:
class AlbuAugmentation:
    def __init__(self):
        ssr_params = dict(
            shift_limit=0.1,
            scale_limit=0.1,
            rotate_limit=10,
            interpolation=3,
            border_mode=cv2.BORDER_CONSTANT,
            p=0.5,
        )

        self.description = [
            albu.OneOf(
                [
                    albu.GaussNoise(p=0.5),
                    albu.MultiplicativeNoise(per_channel=True, p=0.3),
                ],
                p=0.4,
            ),
            albu.OneOf(
                [
                    albu.MotionBlur(blur_limit=3, p=0.2),
                    albu.MedianBlur(blur_limit=3, p=0.2),
                    albu.GaussianBlur(blur_limit=3, p=0.2),
                    albu.Blur(blur_limit=3, p=0.2),
                ],
                p=0.2,
            ),
            albu.OneOf(
                [
                    albu.CLAHE(),
                    albu.Sharpen(),
                    albu.RandomBrightnessContrast(),
                ],
                p=0.3,
            ),
            albu.ShiftScaleRotate(**ssr_params, value=(0,)),
        ]
        self.compose = albu.Compose(self.description, p=1)

    def __call__(self, img: np.ndarray) -> list:
        transformed = self.compose(img)
        return transformed["image"]

## Полезные ссылки

* [Albumentations for image classififcation](https://albumentations.ai/docs/getting_started/image_augmentation/)
* [List of albumentations](https://albumentations.ai/docs/getting_started/transforms_and_targets/)